<a href="https://colab.research.google.com/github/martinpius/Applied-Predictive-Modeling2/blob/master/Ensemble_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Print the correct tensorflow version 

In [1]:
from google.colab import drive
try:
  drive.mount('/content/drive/',force_remount = True)
  COLAB = True
  print("Google colab is on")
  %tensorflow_version 2.x
except:
  COLAB = False
  print("Not using Google Colab")

Mounted at /content/drive/
Google colab is on


In [2]:
#Time formatter
def time_fmt(secs):
  hours = int(secs/(60*60))
  minutes = int(secs%(60*60)/60)
  seconds = int(secs%60)
  return f"{hours}:{minutes:>02}:{seconds:>05.2f}"

In [3]:
#Pertubation implementation to variable importance for NNT

In [5]:
import scipy as sc
import numpy as np
from sklearn import metrics
import math
import pandas as pd


In [7]:
#This function will compute the loss for a regression and classification model
#To provide usefull information in computing variable's importance
#First we select all columns from a test dataset
#Second we randomly shuffles every columns to distort any parttern
#If the fitted model is a regression we compute MSE otherwise we compute log-loss
#We compute variable importance by take the ratio of individual error vs maximum error
#We prepare a results dictionary which is finaly conveted to a dataframe.

In [9]:
def p_rank(model, x, y, names, regression):
  errors = []
  for i in range(x.shape[1]):
    hold = np.array(x[:,i])
    np.random.shuffle(x[:,i])

    if regression:
      preds = model.predict(x)
      err = metrics.mean_squared_error(preds, y)
    else:
      preds = model.predict_proba(x)
      err = metrics.log_loss(preds, y)
    errors.append(err)
    x[:,i] = hold

  max_error = np.max(errors)
  var_importance = [(k/max_error) for k in errors]
  data = {'Name':names, "Errors": errors, "Var_importance": var_importance }
  outputs = pd.DataFrame(data, columns = ['Names', 'Errors', 'Variable Importance'])
  outputs.sort_values(by = ['Variable Importance'], ascending = [0], inplace = True)
  outputs.reset_index(inplace = True, drop = True)
  return outputs

